# <b>MODIS Water Validation Notebook</b>

Purpose: Used to perform validation of C61 MOD44W products. Compares those products to the previous version, C6 MOD44W.

*Note: We are following an incremental development lifecycle. This notebook is the first rendition which fit most of the requirements. Expect incremental releases which continue towards the goal of fully meeting requirements and increasing capabilities of the user.*

Installation requirements:

```bash
pip install localtileserver
```

TODO:
- ipysheet for user to input comments
- load layers from toolbar
- move everything inside a class to avoid user input

Some references:

- https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
- https://github.com/giswqs/geodemo/blob/master/geodemo/common.py

Version: 4.0.0
Date: 07/22/2023

*For DSG internal use*

### <b> WARNING </b>

Do not run all cells at once, doing so will shut down the local tile servers before you, the user, can interact.

Uncomment if localtileserver is not installed

In [ ]:
# !pip install localtileserver

In [ ]:
import os
import re
import json
import joblib
import tempfile
import ipysheet
import numpy as np
import pandas as pd
import rasterio as rio
import rioxarray as rxr
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import ipywidgets as widgets
import warnings
import tempfile

from osgeo import gdal
from pprint import pprint

from glob import glob
from ipysheet import from_dataframe
from localtileserver import TileClient, get_leaflet_tile_layer, examples
from ipyleaflet import Map, Marker, basemaps, ScaleControl, LayersControl, AwesomeIcon
from ipyleaflet import LegendControl, FullScreenControl, MarkerCluster, Popup

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

import localtileserver
from localtileserver import get_leaflet_tile_layer, TileClient

## Tile and year selection

Choose which tile (see MODIS grid) and which year. Reference the grid image. 

The `h` followed by two numerical digits represent the <b>horizontal</b> tile ID. Use the column space to determine this ID. 

The `v` followed by two numerical digits represent the <b>vertical</b> tile ID. Use the row space to determine this ID. 

For example, the tile that is 9 columns to the right and 5 rows down is `h09v05`.

Example:
```python
TILE = 'h09v05'
```

![MODIS Grid Overlay](../imgs/modis_overlay.png)

In [ ]:
TILE = 'h13v02'
# TILE = 'h12v09'
# TILE = 'h21v10'
# TILE = 'h09v05'
# TILE = 'h30v11'
# TILE = 'h28v08'
# TILE = 'h11v02'
# TILE = 'h16v02'
# TILE = 'h17v02'
# TILE = 'h22v01'
# TILE = 'h27v03'
# TILE = 'h18v03'

## NOTICE

Only 2019 is available :D 

In [ ]:
YEAR = 2019

Shouldn't need to change anything under this 

In [ ]:
MOD44W_C6_BASEPATH = '/explore/nobackup/people/mcarrol2/MODIS_water/v5_outputs/'
MOD44W_C61_BASEPATH = '/explore/nobackup/projects/ilab/data/MODIS/PRODUCTION/MODAPS_test3_07202023/MOD44W-LandWaterMask'
MOD44W_C61_VERSION = '001'
C6_FILE_TYPE = '.tif'
C61_FILE_TYPE = '.hdf'

TMP_FILE_TYPE = '.tif'

HDF_PRESTR = 'HDF4_EOS:EOS_GRID'
HDF_POSSTR = 'MOD44W_250m_GRID'

SEVEN_CLASS = 'seven_class'
WATER_MASK = 'water_mask'
WATER_MASK_QA = 'water_mask_QA'

if YEAR > 2019:
    warnings.warn('Using 2019 C6 MOD44W')
    MOD44_C6_YEAR = 2019
else:
    MOD44_C6_YEAR = YEAR

tiles_basemap: str = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}'
water_c6_cmap: list = ['#E3B878', '#2d7d86']
water_c61_cmap: list = ['#194d33', '#8ed1fc']
seven_class_cmap: list = ['#0057d7', '#e6c9a8', '#d700d5', '#00d7d0', '#d70000', '#9f00c7', '#000AD0', '#000564']
# water_qa_cmap: list = ['#FF6900', '#FCB900', '#7BDCB5', '#dd00ff', '#0693E3', '#ff6900', '#EB144C', '#F78DA7', '#9900EF']
water_qa_cmap: list = ['#79d2a6', '#ff6900', '#e4efe9']
difference_cmap: list = ['#b8174e', '#00e202']
water_qa_cmap_dict: dict = {
    1: ('High Confidence Water', '#7BDCB5'),# '#7BDCB5'
    2: ('Low Confidence Water', '#ee82ee'),
    3: ('Low Confidence Land', '#ffe08a'),
    4: ('Ocean Mask', '#FCB900'),
    5: ('Ocean Mask but no water detected', '#0693E3'),
    6: ('Burn Scar (from MCD64A1)', '#FF6900'),
    7: ('Urban/Impervious surface', '#EB144C'),
    8: ('No water detected, Collection 5 shows water', '#F78DA7'),
    9: ('DEM Slope change', '#800080'),
}
CACHE_DIR = '.cache'
os.makedirs(CACHE_DIR, exist_ok=True)

In [ ]:
mod44w_c6_path = os.path.join(MOD44W_C6_BASEPATH, str(MOD44_C6_YEAR), f'MOD44W_{TILE}_{MOD44_C6_YEAR}_v5.tif')
if not os.path.exists(mod44w_c6_path):
    raise FileNotFoundError(f'Could not find the MOD44W C6 file: {mod44w_c6_path}')

In [ ]:
def parse_qa(qa_array: xr.DataArray):
    """
    Parses QA data array for no-data values and
    parses cmap to match present values. Returns
    the parsed QA data array and the cmap list.
    """
    values_to_check = (10, 250, 253, 255)
    qa_array_parsed = xr.where(qa_array == 0, 0, qa_array)
    for value in values_to_check:
        qa_array_parsed = xr.where(qa_array == value, 0, qa_array_parsed)
    values_present = np.unique(qa_array_parsed.data).tolist()
    cmap = []
    for i, value_present in enumerate(values_present):
        if value_present == 0:
            continue
        qa_array_parsed = xr.where(qa_array == value_present, i, qa_array_parsed)
        cmap.append(water_qa_cmap_dict[value_present])
    return qa_array_parsed, cmap
    

In [ ]:
mod44w_c61_regex = os.path.join(MOD44W_C61_BASEPATH,
                     str(YEAR),
                     '001',
                     f'MOD44W.A{YEAR}001.{TILE}.061.*{C61_FILE_TYPE}')

mod44w_c61_path = sorted(glob(mod44w_c61_regex))[0]

print(mod44w_c61_path)

mod44w_c6_data_array = rxr.open_rasterio(mod44w_c6_path)
mod44w_c61_dataset = rxr.open_rasterio(mod44w_c61_path)
mod44w_c61_data_array = mod44w_c61_dataset[WATER_MASK]

mod44w_c61_qa_data_array = mod44w_c61_dataset[WATER_MASK_QA]
mod44w_c61_qa_data_array, qa_data_array_cmap = parse_qa(mod44w_c61_qa_data_array)
qa_data_cmap_colors_only = [hex_val for qa_type, hex_val in qa_data_array_cmap]

mod44w_c61_seven_class_data_array =  mod44w_c61_dataset[SEVEN_CLASS]
mod44w_difference_map = mod44w_c6_data_array.astype(np.int16) - mod44w_c61_data_array.data.astype(np.int16)

In [ ]:
crs = 'PROJCS["Sinusoidal",GEOGCS["Sphere",DATUM["Sphere",SPHEROID["Sphere",6371000,0]],PRIMEM["Greenwich",0],' + \
    'UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"]' + \
    ',PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0]' + \
',UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'
mod44w_cs_ds = gdal.Open(mod44w_c6_path)
transform = mod44w_cs_ds.GetGeoTransform()
mod44w_c6_ds = None

In [ ]:
def open_and_write_temp(data_array, transform, projection, 
                        year, tile, name = None, files_to_rm = None) -> str:
    tmpdir = tempfile.gettempdir()
    name_to_use = data_array.name if not name else name
    tempfile_name = f'MOD44W.A{year}001.{tile}.061.{name_to_use}.tif'
    tempfile_fp = os.path.join(tmpdir, tempfile_name)
    if os.path.exists(tempfile_fp):
        os.remove(tempfile_fp)
    driver = gdal.GetDriverByName('GTiff')
    outDs = driver.Create(tempfile_fp, 4800, 4800, 
                          1, gdal.GDT_Float32, 
                          options=['COMPRESS=LZW'])
    outDs.SetGeoTransform(transform)
    outDs.SetProjection(projection)
    outBand = outDs.GetRasterBand(1)
    outBand.WriteArray(data_array.data[0, :, :])
    outBand.SetNoDataValue(250)
    outDs.FlushCache()
    outDs = None
    outBand = None
    driver = None
    return tempfile_fp

In [ ]:
temporary_files_to_delete = []

mod44w_c61_water_mask = open_and_write_temp(mod44w_c61_data_array, transform, crs, YEAR, TILE, name='c61_mask', files_to_rm=temporary_files_to_delete)
mod44w_c61_water_mask_qa = open_and_write_temp(mod44w_c61_qa_data_array, transform, crs,  YEAR, TILE, name='qa_mask', files_to_rm= temporary_files_to_delete)
mod44w_difference_map = open_and_write_temp(mod44w_difference_map, transform, crs,  YEAR, TILE, name='diff_mask', files_to_rm=temporary_files_to_delete)
mod44w_c61_seven_class = open_and_write_temp(mod44w_c61_seven_class_data_array, transform, crs, YEAR, TILE, name='seven_class', files_to_rm=temporary_files_to_delete)

In [ ]:
mod44w_c6_client = TileClient(mod44w_c6_path)
mod44w_c61_water_client = TileClient(mod44w_c61_water_mask)
mod44w_c61_water_qa_client = TileClient(mod44w_c61_water_mask_qa)
mod44w_difference_client = TileClient(mod44w_difference_map)
mod44w_c61_seven_class_client = TileClient(mod44w_c61_seven_class)

In [ ]:
mod44w_c6_water_mask_layer = get_leaflet_tile_layer(
    mod44w_c6_client, nodata=0, show=False, 
    vmin=0, vmax=1,
    cmap=water_c6_cmap, 
    name=f'MOD44W C6 Water Mask {YEAR} {TILE}',
    max_zoom=20)

mod44w_c61_water_mask_layer = get_leaflet_tile_layer(
    mod44w_c61_water_client, nodata=0, show=False,
    vmin=0, vmax=1,
    cmap=water_c61_cmap, 
    name=f'MOD44W C61 Water Mask {YEAR} {TILE}',
    max_zoom=20)

qa_num_colors = int(mod44w_c61_qa_data_array.max())
mod44w_c61_qa_layer = get_leaflet_tile_layer(
    mod44w_c61_water_qa_client, nodata=0,
    n_colors=qa_num_colors, show=False,
    vmin=1, vmax=qa_num_colors,
    cmap=qa_data_cmap_colors_only, 
    name=f'MOD44W C61 QA Mask {YEAR} {TILE}',
    max_zoom=20)

mod44w_c61_seven_class_layer = get_leaflet_tile_layer(
    mod44w_c61_seven_class_client, nodata=253,
    cmap=seven_class_cmap, n_colors=8,
    vmin=0, vmax=7,
    name=f'MOD44W C61 Seven Class {YEAR} {TILE}',
    max_zoom=20)

mod44w_diference_layer = get_leaflet_tile_layer(
    mod44w_difference_client, nodata=0, show=False,
    vmin=-1, vmax=1,
    cmap=difference_cmap, 
    name=f'MOD44W Difference {YEAR} {TILE}',
    max_zoom=20)

In [ ]:
legend_dict = {}
c61_seven_class_mask_legend_dict = {'Seven Class- Deep Ocean': '#000564', 
                                    'Seven Class- Moderate ocean': '#000AD0',
                                    'Seven Class- Shallow Ocean': '#0057d7',
                                    'Seven Class- Deep Inland Water': '#d70000',
                                    'Seven Class- Inland Water': '#00d7d0',
                                    'Seven Class- Ephemeral Water': '#9f00c7',
                                    'Seven Class- Shoreline': '#d700d5',
                                    'Seven Class- Land': '#e6c9a8'}

c61_qa_mask_legend_dict = {}
for qa_type, hex_val in qa_data_array_cmap:
    c61_qa_mask_legend_dict['C61 QA '+ qa_type] = hex_val

c6_water_mask_legend_dict = {'C6- Water': '#2d7d86'}
c61_water_mask_legend_dict = {'C61- Water': '#8ed1fc'}
mod44w_difference_legend_dict = {'Difference- C61 ONLY Water': '#b8174e', 
                                 'Difference- C6 ONLY Water': '#00e202'}
legend_dict.update(c61_seven_class_mask_legend_dict)
legend_dict.update(c61_qa_mask_legend_dict)
legend_dict.update(c6_water_mask_legend_dict)
legend_dict.update(c61_water_mask_legend_dict)
legend_dict.update(mod44w_difference_legend_dict)

c61_seven_class_mask_legend = LegendControl(legend_dict)

In [ ]:
def get_location(cache_dir: str, tile: str, def_location: list) -> list:
    cache_fp = os.path.join(cache_dir, f'{tile}.marker.location.sv')
    if os.path.exists(cache_fp):
        location = joblib.load(cache_fp)
    else:
        location = def_location
    return location

def cache_location(tile: str, location: list) -> None:
    cache_fp = os.path.join(CACHE_DIR, f'{tile}.marker.location.sv')
    output = joblib.dump(location, cache_fp)
    return None

def initialize_marker(tile: str, location: list, cache_dir: str) -> Marker:
    name = 'Location Marker'
    title = name
    location = get_location(cache_dir, tile, location)
    marker = Marker(name=name, title=name, location=location)
    return marker

def initialize_message(location: list) -> widgets.HTML:
    ll_message = widgets.HTML()
    ll_message.value = str(location)
    return ll_message

In [ ]:
m = Map(
    center=mod44w_c6_client.center(),
    zoom=mod44w_c6_client.default_zoom,
    basemap=basemaps.Esri.WorldImagery,
    scroll_wheel_zoom=True,
    keyboard=True,
    layout=widgets.Layout(height='600px')
)
marker_location = mod44w_c6_client.center()
marker = initialize_marker(tile=TILE, location=marker_location, cache_dir=CACHE_DIR)
latlon_message = initialize_message(marker.location)

def handle_click(**kwargs):
    latlon_message.value = str(marker.location)
    marker.popup = latlon_message
    cache_location(tile=TILE, location=marker.location)

m.add_layer(marker)
marker.on_click(handle_click)
m.add_layer(mod44w_c6_water_mask_layer)
m.add_layer(mod44w_c61_water_mask_layer)
m.add_layer(mod44w_c61_seven_class_layer)
m.add_layer(mod44w_c61_qa_layer)
m.add_layer(mod44w_diference_layer)
m.add_control(c61_seven_class_mask_legend)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(LayersControl(position='topright'))
m.add_control(FullScreenControl())

## MODIS Water Validation Map Visualization

<b>Usage Tips:</b>

- ![Layer Control](../imgs/layer_control.png)    Hover over to select and deselect which layers are visible

- ![Full Screen Control](../imgs/full_screen.png)    Click for full screen

- Use the scroll wheel on the mouse to zoom in and out, or use [+] and [-]

The legend shows all layers no matter what's visible but each element is prefixed with which layer it indicates. I.e.: 

- "Seven Class-": MOD44W C61 Seven Class

- "QA-": MOD44W C61 QA Mask

- "C6-": MOD44W C6 Water Mask

- "C61-": MOD44W C61 Water Mask

- "Difference-": MOD44W Difference

### <b> OPTIONAL CELL </b>

Uncomment and modify lat/lon then run below cell if you want to relocate the marker to a lat/lon of interest quickly.

In [ ]:
#marker.location = [39.20565471434283, -112.55767822265626]

#marker.location = 35.240011164750484, -108.11508178710938

#marker.location = 33.045939492974654, -101.10030878400164

#latlon_message.value = str(marker.location)

#marker.popup = latlon_message

In [ ]:
display(m)

In [ ]:
userid = !whoami
notes_path = f'../notes/{TILE}-{userid[0]}-notes.csv'
if os.path.exists(notes_path):
    notes_df = pd.read_csv(notes_path)
    notes_df = notes_df.drop(columns=['Unnamed: 0'])
    sheet_notes = ipysheet.from_dataframe(notes_df)
else:
    tile = [' ' for _ in range(75)]
    year = [' ' for _ in range(75)]
    location = [' ' for _ in range(75)]
    note = [' ' for _ in range(75)]
    data = {'Tile': tile, 'Year': year, 'Location': location, 'Note': note}
    notes_df = pd.DataFrame(data=data)
    sheet_notes = ipysheet.from_dataframe(notes_df)
sheet_notes.column_width = [3,3,4,10]
sheet_notes.layout = widgets.Layout(width='100%',height='100%')
sheet_notes

## Save notes

Run this cell to save notes in the current working directory

In [ ]:
sheet_notes_df = ipysheet.to_dataframe(sheet_notes)
sheet_notes_df.to_csv(notes_path)

### <b>DO NOT RUN THIS CELL UNTIL FINISHED WITH VALIDATION</b>
*Note: This will shut down the local tile servers*

*Ignore warnings as such:*
```
Server for key (default) not found.
```

In [ ]:
for path_to_delete in temporary_files_to_delete:
    if os.path.exists(path_to_delete):
        os.remove(path_to_delete)
    temporary_files_to_delete.remove(path_to_delete)

mod44w_c6_client.shutdown(True)